In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!ls

artists.csv  sample_data  spark-3.1.1-bin-hadoop3.2  spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [7]:
# Load data from csv to a dataframe.
# header=True means the first row is a header
# sep=';' means the column are seperated using ''
df_artists = spark.read.csv('artists.csv', header=True, sep=",")
df_artists.show(5)
df_artists.count()

+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+----------------+----------------+----------------+
|                mbid|           artist_mb|       artist_lastfm|    country_mb|      country_lastfm|             tags_mb|         tags_lastfm|listeners_lastfm|scrobbles_lastfm|ambiguous_artist|
+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+----------------+----------------+----------------+
|cc197bad-dc9c-440...|            Coldplay|            Coldplay|United Kingdom|      United Kingdom|rock; pop; altern...|rock; alternative...|         5381567|       360111850|           FALSE|
|a74b1b7f-71a5-401...|           Radiohead|           Radiohead|United Kingdom|      United Kingdom|rock; electronic;...|alternative; alte...|         4732528|       499548797|           FALSE|
|8bfac288-ccc5-448...|Red Hot 

594907

In [9]:
#df_artists.columns;
df_artists.printSchema()

root
 |-- mbid: string (nullable = true)
 |-- artist_mb: string (nullable = true)
 |-- artist_lastfm: string (nullable = true)
 |-- country_mb: string (nullable = true)
 |-- country_lastfm: string (nullable = true)
 |-- tags_mb: string (nullable = true)
 |-- tags_lastfm: string (nullable = true)
 |-- listeners_lastfm: string (nullable = true)
 |-- scrobbles_lastfm: string (nullable = true)
 |-- ambiguous_artist: string (nullable = true)



In [11]:
# cleaning and transforming
# removing duplicate values first
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import isnan, when, count, col, lit, trim, avg, ceil
from pyspark.sql.types import StringType
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df_artists = df_artists.dropDuplicates(['mbid'])
print(df_artists.count())

594907


In [12]:
# string cleaning and removing unwanted
str_cols = [item[0] for item in df_artists.dtypes if item[1].startswith('string')]
for cols in str_cols:
  df_artists = df_artists.withColumn(cols, trim(df_artists[cols]))

In [13]:
# lowercasing the columns and put the underscore

# create the temp view table for the sql operations
df_artists.createOrReplaceTempView("artisttbl")

In [14]:
# to understand the data
# define the categories
# 1. artist_mb
# 2. artist_lastfm
# 3. artist_lastf
# 4. country_mb
# 5. country_lastfm
# 6. tags_mb
# 7. ambiguous_artist bool check True or false

# * indicator_code : to represent the valuation indicator code unit name

query_country_counts = '''select row_number() over (order by country_mb asc) rn, country_mb country, count(*) count
from artisttbl art
where art.country_mb is not null
group by country_mb having count(*) > 1'''

query_artist_counts = '''select row_number() over (order by artist_mb asc) rn, country_mb country, artist_mb artist, count(*) count
from artisttbl art
where art.artist_mb is not null and art.country_mb is not null
group by artist_mb,country_mb having count(*) > 1'''

query_clean_known_artists = '''select * from artisttbl art where
ambiguous_artist = FALSE
and tags_mb is not null
and tags_lastfm is not null
and country_mb is not null
and country_lastfm is not null
 '''
clean_all_artists = '''select * from artisttbl art where
 tags_mb is not null
and tags_lastfm is not null
and country_mb is not null
and country_lastfm is not null
 '''
# 36898
# df_known_artists = spark.sql(clean_all_artists)
df_all_artists = spark.sql(clean_all_artists)
df_all_artists.count()

40029

In [32]:
df_all = df_all_artists.toPandas()
df_all.to_csv('all_artists.csv', index = False, encoding='utf-8')

In [ ]:
df_known_artists = df_known_artists.toPandas()
df_all.to_csv('known_artists.csv', index = False, encoding='utf-8')

In [15]:
df_all_artists

mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0008af7d-2aa1-4b4...,Miklós Rózsa,Miklós Rózsa,Hungary,Hungary,soundtrack; class...,Soundtrack; Class...,34430,363238,FALSE
01094f22-a9d8-48f...,The Druids of Sto...,The Druids Of Sto...,United States,United States,psychedelic rock;...,Garage Rock; Psyc...,2009,17987,FALSE
01e1aae4-5148-402...,Avanti!,Avanti!,Finland,Finland,classical; chambe...,seen live; skweee...,841,4623,FALSE
02e5d1fe-db84-450...,Seun Kuti,Seun Kuti,Nigeria,Nigeria; Poland,world; african; a...,afrobeat; seen li...,25713,119345,FALSE
02fc3032-e491-46b...,Old Man’s Child,Old Man's Child,Norway,Norway,norwegian,black metal; melo...,132320,3575801,FALSE
05ab8680-38d8-449...,柏木広樹,柏木広樹,Japan,Japan,likedis auto,cello; Smooth Jaz...,249,21838,FALSE
05b82580-9121-4d9...,Russell Mills,Russell Mills,United Kingdom,United Kingdom,ambient; british;...,ambient; electron...,3073,18955,FALSE
080157d9-fe06-43e...,Adamski,Adamski,United Kingdom,United Kingdom,house; british; u...,electronic; dance...,111268,467948,FALSE
0a77bec1-12ef-4ca...,Bent,Bent,United Kingdom,United Kingdom,dance and electro...,chillout; electro...,562269,6502551,FALSE
0be75788-c49b-474...,Projected Twin,Projected Twin,Australia,Australia,progressive rock;...,Progressive rock;...,1684,37261,FALSE


In [ ]:
# transform RDDs